In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Bidirectional
from tensorflow.keras.optimizers import Adam

In [5]:
data=open("./tmp/irish.txt").read()

tokenizer=Tokenizer()
corpus=data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words=len(tokenizer.word_index)+1
print(tokenizer.word_index)
print(total_words)

{'gain': 2454, 'clouds': 1529, 'apple': 1719, 'tiny': 1110, 'dropped': 2373, 'linnet': 1902, 'lusty': 1221, 'james': 1583, 'unpaid': 2160, 'limbs': 1999, 'mã¯â¿â½r': 2177, 'out': 68, 'kinds': 1828, 'kicked': 1875, 'treated': 1780, 'over': 84, 'oconnell': 1479, 'prove': 1001, 'odds': 2074, 'iull': 1691, 'boyne': 940, 'banter': 1933, 'bond': 1321, 'sociable': 1787, 'foam': 1395, 'where': 47, 'breathes': 1230, 'rosy': 630, 'bottle': 2337, 'native': 910, 'valley': 432, 'distressful': 2610, 'tears': 314, 'strife': 998, 'rode': 1312, 'travelled': 2244, 'grove': 1205, 'whispering': 1419, 'offer': 1606, 'blame': 1143, 'rolling': 1740, 'tralee': 791, 'brooks': 1840, 'should': 224, 'fades': 2600, 'cornwall': 2266, 'coffee': 2097, 'letters': 1003, 'lingers': 2568, 'whisperings': 1442, 'courting': 1047, 'strangely': 1255, 'able': 319, 'rays': 709, 'west': 656, 'horse': 477, 'case': 1352, 'feels': 2128, 'song': 212, 'damer': 1945, 'poured': 1596, 'top': 2480, 'enthralled': 1914, 'someones': 2123, '

In [14]:
input_sequence=[]
for line in corpus:
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequence.append(n_gram_sequence)

max_sequence_len=max([len(x) for x in input_sequence])
input_sequence=np.array(pad_sequences(input_sequence,maxlen=max_sequence_len,padding="pre"))
xs,labels=input_sequence[:,:-1],input_sequence[:,-1]
ys=tf.keras.utils.to_categorical(labels,num_classes=total_words)

In [15]:
print(xs[897])
print(labels[34])
print(ys[9])

[  0   0   0   0   0   0   0   0   0   0 485 243  72 124  28]
801
[0. 0. 0. ... 0. 0. 0.]


In [16]:
print(ys.shape)

(12038, 2690)


In [17]:
model=Sequential([
    Embedding(total_words,100,input_length=max_sequence_len-1),
    Bidirectional(LSTM(150)),
    Dense(total_words,activation="softmax")
])

adam=Adam(lr=0.01)
model.compile(loss="categorical_crossentropy",optimizer=adam,metrics=["accuracy"])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history=model.fit(xs,ys,epochs=100,verbose=1)

print(model)

Epoch 1/100
377/377 [==============================] - 19s 51ms/step - accuracy: 0.0748 - loss: 6.6546
Epoch 2/100
377/377 [==============================] - 18s 48ms/step - accuracy: 0.1141 - loss: 5.7610
Epoch 3/100
377/377 [==============================] - 18s 47ms/step - accuracy: 0.1571 - loss: 4.9171
Epoch 4/100
377/377 [==============================] - 18s 48ms/step - accuracy: 0.2235 - loss: 4.0506
Epoch 5/100
377/377 [==============================] - 20s 52ms/step - accuracy: 0.3176 - loss: 3.2324
Epoch 6/100
377/377 [==============================] - 18s 48ms/step - accuracy: 0.4209 - loss: 2.5729
Epoch 7/100
377/377 [==============================] - 18s 49ms/step - accuracy: 0.5249 - loss: 2.0305
Epoch 8/100
377/377 [==============================] - 19s 49ms/step - accuracy: 0.6033 - loss: 1.6526
Epoch 9/100
377/377 [==============================] - 19s 51ms/step - accuracy: 0.6615 - loss: 1.4062
Epoch 10/100
377/377 [==============================] - 19s 50ms/step - a

377/377 [==============================] - 17s 44ms/step - accuracy: 0.7372 - loss: 0.9850
Epoch 80/100
377/377 [==============================] - 16s 43ms/step - accuracy: 0.7486 - loss: 0.9767
Epoch 81/100
377/377 [==============================] - 16s 44ms/step - accuracy: 0.7401 - loss: 1.0269
Epoch 82/100
377/377 [==============================] - 17s 46ms/step - accuracy: 0.7520 - loss: 0.9615
Epoch 83/100
377/377 [==============================] - 15s 40ms/step - accuracy: 0.7730 - loss: 0.8689
Epoch 84/100
377/377 [==============================] - 15s 41ms/step - accuracy: 0.7920 - loss: 0.7944
Epoch 85/100
377/377 [==============================] - 15s 41ms/step - accuracy: 0.7943 - loss: 0.7732
Epoch 86/100
377/377 [==============================] - 16s 42ms/step - accuracy: 0.7935 - loss: 0.7823
Epoch 87/100
377/377 [==============================] - 17s 44ms/step - accuracy: 0.7736 - loss: 0.88620s - accur
Epoch 88/100
377/377 [==============================] - 16s 44ms/st

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["accuracy"])
plt.xlabel("Epochs")
plt.ylabel("accuracy")
plt.show()

In [ ]:
seed_text="If only i could tell how those wind mills "
next_words=20

for _ in range(next_words):
    token_list=tokenizer.text_to_sequences([seed_text])[0]
    token_list=pad_sequences([token_list],maxlen=max_sequence_len,padding="pre")
    predicted=model.predict_classes(token_list,verbose=0)
    output_word=""
    for word,index in tokenizer.word_index.items():
        if index==predicted:
            output_word=word
            break
    seed_text+=output_word
print(seed_text)
    
    
    
    
    
    
    